In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

In [2]:
# 적용인구 현황
inf = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='82p_인구')

# 1. 3번 행을 칼럼명으로 변경
inf.columns = inf.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf = inf.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf = inf.drop(index=[56, 57, 58])

# 인덱스 리셋
inf = inf.reset_index(drop=True)

inf.iloc[:, [2, 3]] = inf.iloc[:, [3, 2]].values
inf.iloc[:, [6, 7]] = inf.iloc[:, [7, 6]].values

inf = inf.loc[:, ~inf.columns.isna()]

inf = inf.drop(columns = ["G. & T.", "Division"])

inf.columns = [col.split('\n')[0] for col in inf.columns]
inf.columns = [col.split('  C.E')[0] for col in inf.columns]
inf.columns = [col.split('  S.E.')[0] for col in inf.columns]
inf.columns = [col.split('    Medical Aid')[0] for col in inf.columns]

inf

# 창원시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf[inf['구  분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 창원시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf.columns)

# 전주시 행 추가 및 기존 구 삭제
inf = pd.concat([inf[~inf['구  분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf = inf.rename(columns={'구  분': '지역'})

inf['지역'] = inf['지역'] + ' ' + inf.groupby('지역').cumcount().replace(0, '').astype(str)

inf

,지역,의료보장,근 로 자,공ㆍ교,지 역,의료급여 1종,의료급여 2종
0,광주,1472535,900866,141552,366687,40646,22784
1,동구,98872,54224,11494,27146,4141,1867
2,서구,303357,184831,30815,74963,8159,4589
3,남구,218884,128640,25974,54731,6110,3429
4,북구,436493,259429,42036,113714,13816,7498
5,광산구,414929,273742,31233,96133,8420,5401
6,전북,1837478,1048946,194740,508751,60362,24679
7,군산시,273214,163835,22011,75141,8920,3307
8,익산시,290541,169903,26017,79827,10782,4012
9,정읍시,111951,58367,11896,35142,4825,1721


In [3]:
# 연령별 의료 보장
inf2 = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='84p')

# 1. 3번 행을 칼럼명으로 변경
inf2.columns = inf2.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf2 = inf2.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 56, 57, 58번 행 삭제
inf2 = inf2.drop(index=[56, 57])

# 인덱스 리셋
inf2 = inf2.reset_index(drop=True)

inf2 = inf2.drop(columns = [ "Division"])

inf2.columns = [col.split('\n')[0] for col in inf2.columns]

# 전주시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf2[inf2['구  분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 전주시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf2.columns)

# 전주시 행 추가 및 기존 구 삭제
inf2 = pd.concat([inf2[~inf2['구  분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf2 = inf2.rename(columns={'구  분': '지역'})

inf2['지역'] = inf2['지역'] + ' ' + inf2.groupby('지역').cumcount().replace(0, '').astype(str)

inf2

,지역,계,0-4세,5-9세,10-14세,15-19세,20-24세,25-29세,30-34세,35-39세,40-44세,45-49세,50-54세,55-59세,60-64세,65-69세,70-74세,75-79세,80세 이상
0,광주,1472535,54358,72968,74707,90078,110759,101523,85816,111737,114891,132662,123572,112368,92441,61698,50869,39429,42659
1,동구,98872,3195,3478,3051,4443,6929,7087,6000,6762,6299,7340,7386,8071,7964,5976,5447,4574,4870
2,서구,303357,9829,13708,14754,18950,23959,21960,17520,21921,23001,27974,26501,23714,19428,12970,10479,8020,8669
3,남구,218884,7493,10860,10953,12732,15401,13570,11473,15955,16370,18318,17167,16871,15813,11182,9574,7303,7849
4,북구,436493,15629,19673,20035,25273,32556,31020,26130,32237,31740,37410,37273,36102,30116,20092,16088,12212,12907
5,광산구,414929,18212,25249,25914,28680,31914,27886,24693,34862,37481,41620,35245,27610,19120,11478,9281,7320,8364
6,전북,1837478,57825,79732,82719,99975,118271,102451,88947,115084,123868,153372,152073,154507,139553,104194,87547,76764,100596
7,군산시,273214,9305,13339,13017,14843,16550,14422,13976,19920,20510,24434,22168,22365,20594,15124,12133,9490,11024
8,익산시,290541,8813,12184,13118,16465,20907,17591,13947,17596,19295,24491,25036,25449,22405,16026,12663,10695,13860
9,정읍시,111951,2839,4087,4549,5737,6551,5086,4406,5571,6802,8771,9361,9735,9203,7041,6710,6507,8995


In [4]:
# 요양기관 현황
inf3 = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='90p심평원')

# 1. 3번 행을 칼럼명으로 변경
inf3.columns = inf3.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf3 = inf3.drop(index=[0, 1, 2, 3, 4])

# 3. 46, 47, 48번 행 삭제
inf3 = inf3.drop(index=[53, 54, 55])

# 인덱스 리셋
inf3 = inf3.reset_index(drop=True)

inf3.columns = [col.split('\n')[0] for col in inf3.columns]

inf3 = inf3.drop(columns = [ "조  산  원", "Division"])
inf3

# 전주시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf3[inf3['구      분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 전주시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf3.columns)

# 전주시 행 추가 및 기존 구 삭제
inf3 = pd.concat([inf3[~inf3['구      분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf3 = inf3.rename(columns={'구      분': '지역'})

# 예시: 'col1', 'col2', 'col3'의 값을 합쳐 'new_col' 생성
inf3['보건소'] = inf3[['보  건  소', '보건지소','보건진료소', '보건의료원']].sum(axis=1)

inf3 = inf3.drop(columns = [  '보  건  소', '보건지소','보건진료소', '보건의료원'])

inf3 = inf3.rename(columns={'구      분': '지역'})

inf3['지역'] = inf3['지역'] + ' ' + inf3.groupby('지역').cumcount().replace(0, '').astype(str)

inf3

,지역,계,상급종합병원,종합병원,병 원,요양병원,의 원,치과병원,치과의원,한방병원,한 의 원,약 국,보건소
0,광주,2852,2,21,80,66,948,14,625,85,323,672,16
1,동구,327,2,1,6,10,113,1,56,6,34,97,1
2,서구,676,0,5,19,11,250,9,144,18,87,131,2
3,남구,427,0,3,15,9,128,0,103,17,45,104,3
4,북구,782,0,6,22,19,251,1,183,26,85,188,1
5,광산구,640,0,6,18,17,206,3,139,18,72,152,9
6,전북,3774,2,11,74,83,1165,3,588,30,505,908,405
7,군산시,494,0,2,6,9,159,0,77,2,74,131,34
8,익산시,556,1,1,7,11,174,1,95,5,78,143,40
9,정읍시,256,0,1,4,5,76,0,34,1,32,61,42


In [5]:
# 의료인력현황
inf4 = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='92p')

# 1. 3번 행을 칼럼명으로 변경
inf4.columns = inf4.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf4 = inf4.drop(index=[0, 1, 2, 3, 4, 5])

# 3. 46, 47, 48번 행 삭제
inf4 = inf4.drop(index=[54, 55, 56])

# 인덱스 리셋
inf4 = inf4.reset_index(drop=True)
inf4 = inf4.loc[:, ~inf4.columns.isna()]

inf4.columns = [col.split('\n')[0] for col in inf4.columns]
inf4.columns = [col.split('physical')[0] for col in inf4.columns]
inf4 = inf4.drop(columns = [ "Division"])

# 전주시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf4[inf4['구   분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 전주시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf4.columns)

# 전주시 행 추가 및 기존 구 삭제
inf4 = pd.concat([inf4[~inf4['구   분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf4 = inf4.rename(columns={'구   분': '지역'})

inf4['지역'] = inf4['지역'] + ' ' + inf4.groupby('지역').cumcount().replace(0, '').astype(str)

inf4

,지역,계,의 사,치 과 의 사,한 의 사,간호사,약 사,물리치료사,작업치료사,사회복지사
0,광주,17835,3658,1009,723,8949,1276,1734,309,177
1,동구,3895,1005,171,62,2262,202,142,21,30
2,서구,3139,681,209,189,1390,242,334,56,38
3,남구,2796,496,127,141,1461,189,299,60,23
4,북구,4106,747,294,178,1827,333,565,111,51
5,광산구,3899,729,208,153,2009,310,394,61,35
6,전북,16821,3689,917,934,7652,1427,1730,287,185
7,군산시,2018,405,115,118,873,193,239,57,18
8,익산시,3043,654,179,141,1463,233,283,63,27
9,정읍시,746,157,41,58,311,86,81,4,8


In [6]:
#주요시설현황
inf5 = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='94p')

# 1. 3번 행을 칼럼명으로 변경
inf5.columns = inf5.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf5 = inf5.drop(index=[0, 1, 2, 3, 4, 5, 6])

# 3. 46, 47, 48번 행 삭제
inf5 = inf5.drop(index=[55, 56, 57])

# 인덱스 리셋
inf5 = inf5.reset_index(drop=True)

inf5.iloc[:, [3, 4]] = inf5.iloc[:, [4, 3]].values
inf5.iloc[:, [15, 16]] = inf5.iloc[:, [16, 15]].values

inf5 = inf5.loc[:, ~inf5.columns.isna()]

inf5.columns = [col.split('\n')[0] for col in inf5.columns]
inf5.columns = [col.split(' Inpatient room')[0] for col in inf5.columns]
inf5.columns = [col.split('Intensive Care Unit')[0] for col in inf5.columns]
inf5 = inf5.drop(columns =  [ '계',  '정신과폐쇄',
'격리병실                           Isolation room', '무균치료실                   Aseptic room', "Division"])

# 전주시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf5[inf5['구   분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 전주시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf5.columns)

# 전주시 행 추가 및 기존 구 삭제
inf5 = pd.concat([inf5[~inf5['구   분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf5 = inf5.rename(columns={'구   분': '지역'})

inf5['지역'] = inf5['지역'] + ' ' + inf5.groupby('지역').cumcount().replace(0, '').astype(str)

inf5.columns

Index(['지역', '일반입원실 ', '중환자실  '], dtype='object')

In [7]:
# 의료장비 현황
inf6 = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='96p')

# 1. 3번 행을 칼럼명으로 변경
inf6.columns = inf6.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf6 = inf6.drop(index=[0, 1, 2, 3, 4])

# 3. 46, 47, 48번 행 삭제
inf6 = inf6.drop(index=[53, 54, 55])

# 인덱스 리셋
inf6 = inf6.reset_index(drop=True)

inf6.columns = [col.split('\n')[0] for col in inf6.columns]
inf6.columns = [col.split(' Inpatient room')[0] for col in inf6.columns]
inf6.columns = [col.split('Intensive Care Unit')[0] for col in inf6.columns]
inf6 = inf6.drop(columns = [  '계', '유         방',
       '양전자 방사                                 단층 촬영                            PET',
       '체외충격파쇄석기     Extracorporead', '혈관조영장치', '혈관조영장치', 'Division'])

# 전주시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf6[inf6['구   분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 전주시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf6.columns)

# 전주시 행 추가 및 기존 구 삭제
inf6 = pd.concat([inf6[~inf6['구   분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf6 = inf6.rename(columns={'구   분': '지역'})

inf6['지역'] = inf6['지역'] + ' ' + inf6.groupby('지역').cumcount().replace(0, '').astype(str)

inf6

,지역,자기공명영상기,전산화단층촬영장치,초음파
0,광주,74,80,1185
1,동구,8,13,182
2,서구,22,21,265
3,남구,9,10,170
4,북구,16,18,301
5,광산구,19,18,267
6,전북,62,97,1211
7,군산시,9,11,155
8,익산시,8,15,209
9,정읍시,2,5,63


In [8]:
# 진료실적현황
inf7 = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='118p')

# 1. 3번 행을 칼럼명으로 변경
inf7.columns = inf7.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf7 = inf7.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 56,57번 행 삭제
inf7 = inf7.drop(index=[56, 57])

# 인덱스 리셋
inf7 = inf7.reset_index(drop=True)

inf7 = inf7.loc[:, ~inf7.columns.isna()]

inf7.columns = [col.split('\n')[0] for col in inf7.columns]
inf7 = inf7.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])


# 전주시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf7[inf7['구 분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 전주시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf7.columns)

# 전주시 행 추가 및 기존 구 삭제
inf7 = pd.concat([inf7[~inf7['구 분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf7 = inf7.rename(columns={'구 분': '지역'})

inf7['지역'] = inf7['지역'] + ' ' + inf7.groupby('지역').cumcount().replace(0, '').astype(str)

inf7

,지역,진료실인원,진료비,급여비
0,광주,1391896,2820451155.79,2186377062.098
1,동구,92949,223787232.51,175054503.482
2,서구,286684,573460847.41,442767025.515
3,남구,207626,449452064.61,347834409.375
4,북구,412750,871168475.46,679587608.672
5,광산구,391887,702582535.8,541133515.054
6,전북,1749441,4096643466.93,3212230898.82
7,군산시,259590,554985782.18,435952469.532
8,익산시,276015,614863781.69,483397875.484
9,정읍시,106475,291731649.8,231566753.082


In [9]:
# 진료실적 관내
inf8 = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='128p')


# 1. 3번 행을 칼럼명으로 변경
inf8.columns = inf8.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf8 = inf8.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf8 = inf8.drop(index=[56, 57])

# 인덱스 리셋
inf8 = inf8.reset_index(drop=True)

inf8 = inf8.loc[:, ~inf8.columns.isna()]

inf8.columns = [col.split('\n')[0] for col in inf8.columns]
inf8 = inf8.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])

# 전주시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf8[inf8['구 분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 전주시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf8.columns)

# 전주시 행 추가 및 기존 구 삭제
inf8 = pd.concat([inf8[~inf8['구 분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf8 = inf8.rename(columns={'구 분': '지역'})

inf8['지역'] = inf8['지역'] + ' ' + inf8.groupby('지역').cumcount().replace(0, '').astype(str)

inf8

,지역,진료실인원,진료비,급여비
0,광주,1351058,2379347705.33,1819159415.155
1,동구,73499,96641527.13,75417685.176
2,서구,253597,241093687.76,181858011.135
3,남구,177003,189214642.04,143938515.461
4,북구,364479,419891784.94,323144131.948
5,광산구,357705,387310853.63,292088399.305
6,전북,1678061,3426809582.28,2674202603.868999
7,군산시,243962,367868647.42,285151273.069
8,익산시,256862,466299507.4,363653945.664
9,정읍시,95780,155319547.7,122368648.542


In [10]:
# 관외
inf9 = pd.read_excel("03_시군구별 제3권-광주 전북 전남 제주_2019년.xlsx",sheet_name='134p')

# 1. 3번 행을 칼럼명으로 변경
inf9.columns = inf9.iloc[3]

# 2. 0, 1, 2, 4번 행 삭제
inf9 = inf9.drop(index=[0, 1, 2, 3, 4, 5, 6, 7])

# 3. 46, 47, 48번 행 삭제
inf9 = inf9.drop(index=[56, 57])
inf9

# 인덱스 리셋
inf9 = inf9.reset_index(drop=True)

inf9 = inf9.loc[:, ~inf9.columns.isna()]


inf9.columns = [col.split('\n')[0] for col in inf9.columns]
inf9 = inf9.drop(columns = ['내원일수', '급여일수',
       '진료실인원당     Per Patient', '내원1일당  Per Visit ',
       '급여1일당  Per reimbursed day', 'Division'])

# 전주시 관련 행 필터링 및 합계 계산
jeonju =[ '전주시완산구             ','전주시덕진구             ']
jeonju_data = inf9[inf9['구 분'].isin(jeonju)].iloc[:, 1:].sum()

# 새로운 전주시 행 생성
jeonju_row = pd.DataFrame([['전주시'] + jeonju_data.tolist()], columns=inf9.columns)

# 전주시 행 추가 및 기존 구 삭제
inf9 = pd.concat([inf9[~inf9['구 분'].isin(jeonju)], jeonju_row], ignore_index=True)
inf9 = inf9.rename(columns={'구 분': '지역'})

inf9['지역'] = inf9['지역'] + ' ' + inf9.groupby('지역').cumcount().replace(0, '').astype(str)

inf9

,지역,진료실인원,진료비,급여비
0,광주,322314,441103450.46,367217646.943
1,동구,79029,127145705.38,99636818.306
2,서구,205470,332367159.65,260909014.38
3,남구,158932,260237422.57,203895893.914
4,북구,289860,451276690.52,356443476.724
5,광산구,233307,315271682.17,249045115.749
6,전북,412199,669833884.65,538028294.951
7,군산시,107002,187117134.76,150801196.463
8,익산시,103802,148564274.29,119743929.82
9,정읍시,62075,136412102.1,109198104.54


In [11]:
dataframes = [inf, inf2, inf3, inf4, inf5, inf6, inf7, inf8, inf9]  # inf4~inf9를 추가하세요

# 순차적으로 merge 진행
merged_df = dataframes[0]
for df in dataframes[1:]:
    merged_df = pd.merge(merged_df, df, on='지역')

merged_df = merged_df.drop_duplicates(subset='지역', keep='first')

merged_df

,지역,의료보장,근 로 자,공ㆍ교,지 역,의료급여 1종,의료급여 2종,계_x,0-4세,5-9세,...,초음파,진료실인원_x,진료비_x,급여비_x,진료실인원_y,진료비_y,급여비_y,진료실인원,진료비,급여비
0,광주,1472535,900866,141552,366687,40646,22784,1472535,54358,72968,...,1185,1391896,2820451155.79,2186377062.098,1351058,2379347705.33,1819159415.155,322314,441103450.46,367217646.943
1,동구,98872,54224,11494,27146,4141,1867,98872,3195,3478,...,182,92949,223787232.51,175054503.482,73499,96641527.13,75417685.176,79029,127145705.38,99636818.306
2,서구,303357,184831,30815,74963,8159,4589,303357,9829,13708,...,265,286684,573460847.41,442767025.515,253597,241093687.76,181858011.135,205470,332367159.65,260909014.38
3,남구,218884,128640,25974,54731,6110,3429,218884,7493,10860,...,170,207626,449452064.61,347834409.375,177003,189214642.04,143938515.461,158932,260237422.57,203895893.914
4,북구,436493,259429,42036,113714,13816,7498,436493,15629,19673,...,301,412750,871168475.46,679587608.672,364479,419891784.94,323144131.948,289860,451276690.52,356443476.724
5,광산구,414929,273742,31233,96133,8420,5401,414929,18212,25249,...,267,391887,702582535.8,541133515.054,357705,387310853.63,292088399.305,233307,315271682.17,249045115.749
6,전북,1837478,1048946,194740,508751,60362,24679,1837478,57825,79732,...,1211,1749441,4096643466.93,3212230898.82,1678061,3426809582.28,2674202603.868999,412199,669833884.65,538028294.951
7,군산시,273214,163835,22011,75141,8920,3307,273214,9305,13339,...,155,259590,554985782.18,435952469.532,243962,367868647.42,285151273.069,107002,187117134.76,150801196.463
8,익산시,290541,169903,26017,79827,10782,4012,290541,8813,12184,...,209,276015,614863781.69,483397875.484,256862,466299507.4,363653945.664,103802,148564274.29,119743929.82
9,정읍시,111951,58367,11896,35142,4825,1721,111951,2839,4087,...,63,106475,291731649.8,231566753.082,95780,155319547.7,122368648.542,62075,136412102.1,109198104.54


In [12]:
file_path = "2019 전라 제주.csv"  # 저장할 파일 이름과 경로 설정
merged_df.to_csv(file_path, index=False, encoding='utf-8-sig')  # UTF-8 인코딩으로 저장